<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Victor Faraggi V.

### **Link de repositorio de GitHub:** `https://github.com/stepp1/MDS7202-2021`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
# !pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
# !pip install umap-learn

# Librería para NLP
# %conda install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_comics = pd.read_csv('df_comics.csv', index_col=0)
df_comics_no_label = pd.read_csv('comics_no_label.csv', index_col=0)
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
df_comics_no_label.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
14,Ace Morgan,Kyle Morgan,NaN,7,NaN,"Aviation: Ace is an extremely skilled pilot, ...",85,20,65,55,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
17,A'dal,NaN,NaN,7,"As with most of the naaru, little is known of ...",NaN,85,30,70,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,Agent Zero (FOX),David North,NaN,6,"During mid-late 1973, Zero was a member of Tea...",Zero can absorb kinetic energy to further incr...,90,10,25,25,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
31,Ajax (FOX),Francis,Francis Freeman,7,Ajax (born Francis Freeman) was a human who ga...,Ajax has claimed that the procedure to obtain ...,85,25,45,75,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0


In [6]:
print(df_comics.isna().sum(), "\n\n===\n\n", df_comics_no_label.isna().sum())

name                    0
real_name             121
full_name             390
overall_score           0
history_text            0
                     ... 
has_super_speed        57
has_durability         57
has_stamina            57
has_agility            57
has_super_strength     57
Length: 81, dtype: int64 

===

 name                   2
real_name             18
full_name             71
overall_score          0
history_text           8
                      ..
has_super_speed        6
has_durability         6
has_stamina            6
has_agility            6
has_super_strength     6
Length: 81, dtype: int64


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [7]:
docs = ['The teacher rocks like a good rock & roll', 'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [8]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [9]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [10]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [15]:
#### Código aquí ####
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

pipe_preproc = Pipeline([
    ("preprocess", ColumnTransformer(transformers=[
        ("bow", CountVectorizer(tokenizer=StemmerTokenizer(), ngram_range=(1,2)), "history_text"),
        ("minmax", MinMaxScaler(), atributos_de_interes),
    ]))
])

processed =  pipe_preproc.fit_transform(df_comics)
processed.shape

(1285, 333926)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [16]:
#### Código aquí ####

pipe_nb = Pipeline([
    ("preprocess", pipe_preproc),
    ("feat-selector", SelectPercentile(score_func=f_classif, percentile=90)),
    ("clf", MultinomialNB())
    
])

seed = 84

X, y = df_comics[[col for col in df_comics.columns if col != "alignment"]], df_comics["alignment"].to_numpy().ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

fitted_nb = pipe_nb.fit(X_train, y_train)

In [17]:
y_pred = fitted_nb.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.73      0.06      0.12       124
        Good       0.61      0.99      0.75       229
     Neutral       1.00      0.03      0.06        33

    accuracy                           0.61       386
   macro avg       0.78      0.36      0.31       386
weighted avg       0.68      0.61      0.49       386



## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [38]:
#### Código aquí ####
pipe_rf = Pipeline([
    ("preproc", pipe_preproc),
    ("feat-selector", SelectPercentile(score_func=f_classif)),
    ("clf", RandomForestClassifier(max_depth=100, random_state=seed))
    
])

pipe_logreg = Pipeline([
    ("preproc", pipe_preproc),
    ("feat-selector", SelectPercentile(score_func=f_classif)),
    ("clf", LogisticRegression(l1_ratio=0.5, max_iter=1000, random_state=seed))
    
])


logreg_params = {
    "preproc__preprocess__bow__ngram_range": [(1,1), (1,2), (1,3)],
    "feat-selector__percentile": [30, 60, 80],
    "clf__C": [0.5, 1.0, 5.],
    "clf__penalty": ["l2", "elasticnet"],
    "clf__solver": ["saga"]
}

rf_params = {
    "preproc__preprocess__bow__ngram_range": [(1,1), (1,2), (1,3)],
    "feat-selector__percentile": [30, 60, 80],
    "clf__n_estimators": [50, 100, 250],
    "clf__criterion": ["gini", "entropy"],
    "clf__max_features": ["sqrt", "log2"]
}

In [39]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

search_logreg = HalvingGridSearchCV(pipe_logreg, logreg_params, cv=3, verbose=10)
search_rf = HalvingGridSearchCV(pipe_rf, rf_params, cv=3, verbose=10)

In [40]:
search_logreg.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 33
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 33
Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV 1/3; 1/54] START clf__C=0.5, clf__penalty=l2, clf__solver=saga, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)


/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/3; 1/54] END clf__C=0.5, clf__penalty=l2, clf__solver=saga, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.636) total time=   0.6s
[CV 2/3; 1/54] START clf__C=0.5, clf__penalty=l2, clf__solver=saga, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)


/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1476: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/3; 1/54] END clf__C=0.5, clf__penalty=l2, clf__solver=saga, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.364) total time=   0.6s
[CV 3/3; 1/54] START clf__C=0.5, clf__penalty=l2, clf__solver=saga, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)


...

[CV 2/3; 2/2] END clf__C=0.5, clf__penalty=elasticnet, clf__solver=saga, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=0.806, test=0.663) total time=  55.3s
[CV 3/3; 2/2] START clf__C=0.5, clf__penalty=elasticnet, clf__solver=saga, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1)


/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 3/3; 2/2] END clf__C=0.5, clf__penalty=elasticnet, clf__solver=saga, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=0.773, test=0.659) total time=  55.4s


/home/step/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('preproc',
                                               Pipeline(steps=[('preprocess',
                                                                ColumnTransformer(transformers=[('bow',
                                                                                                 CountVectorizer(ngram_range=(1,
                                                                                                                              2),
                                                                                                                 tokenizer=<__main__.StemmerTokenizer object at 0x7f1acb92dca0>),
                                                                                                 'history_text'),
                                                                                                ('minmax',
                                                                            

In [41]:
search_logreg.best_estimator_

Pipeline(steps=[('preproc',
                 Pipeline(steps=[('preprocess',
                                  ColumnTransformer(transformers=[('bow',
                                                                   CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x7f1acb5d46a0>),
                                                                   'history_text'),
                                                                  ('minmax',
                                                                   MinMaxScaler(),
                                                                   ['intelligence_score',
                                                                    'strength_score',
                                                                    'speed_score',
                                                                    'durability_score',
                                                                    'power_score',
                            

In [56]:
search_logreg.score(X_test, y_test)

0.6632124352331606

In [51]:
search_rf.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 5
n_possible_iterations: 4
min_resources_: 18
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 108
n_resources: 18
Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV 1/3; 1/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 1/3; 1/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.667) total time=   0.5s
[CV 2/3; 1/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 2/3; 1/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.333) 

[CV 1/3; 8/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 2);, score=(train=1.000, test=0.167) total time=   0.5s
[CV 2/3; 8/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 2)
[CV 2/3; 8/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 2);, score=(train=1.000, test=0.500) total time=   0.4s
[CV 3/3; 8/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 2)
[CV 3/3; 8/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=50, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 2);, score=(train=1.000, test=0.200) total time=   0.6s
[CV 1/3; 9/108] START clf__

[CV 2/3; 15/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.333) total time=   0.5s
[CV 3/3; 15/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 3/3; 15/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.200) total time=   0.6s
[CV 1/3; 16/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 1/3; 16/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.333) total time=   0.5s
[CV 2/3; 16/108] 

[CV 3/3; 22/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.200) total time=   0.7s
[CV 1/3; 23/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 2)
[CV 1/3; 23/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 2);, score=(train=1.000, test=0.667) total time=   0.8s
[CV 2/3; 23/108] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 2)
[CV 2/3; 23/108] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 2);, score=(train=1.000, test=0.333) total time=   0.7s
[CV 3/3; 23/108] 

[CV 1/3; 30/108] END clf__criterion=gini, clf__max_features=log2, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.667) total time=   0.6s
[CV 2/3; 30/108] START clf__criterion=gini, clf__max_features=log2, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 2/3; 30/108] END clf__criterion=gini, clf__max_features=log2, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.333) total time=   0.5s
[CV 3/3; 30/108] START clf__criterion=gini, clf__max_features=log2, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 3/3; 30/108] END clf__criterion=gini, clf__max_features=log2, clf__n_estimators=50, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.200) total time=   0.5s
[CV 1/3; 31/108] START

...

[CV 2/3; 24/36] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.667) total time=   0.6s
[CV 3/3; 24/36] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 3/3; 24/36] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.647) total time=   1.0s
[CV 1/3; 25/36] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 1/3; 25/36] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.667) total time=   1.2s
[CV 2/3; 25/36] START 

[CV 3/3; 31/36] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.412) total time=   1.2s
[CV 1/3; 32/36] START clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 1/3; 32/36] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.667) total time=   1.5s
[CV 2/3; 32/36] START clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 2/3; 32/36] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.667) total time=   1.0s
[CV 3/3

[CV 1/3; 3/12] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.630) total time=   3.4s
[CV 2/3; 3/12] START clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 2/3; 3/12] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.630) total time=   4.0s
[CV 3/3; 3/12] START clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3)
[CV 3/3; 3/12] END clf__criterion=entropy, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=60, preproc__preprocess__bow__ngram_range=(1, 3);, score=(train=1.000, test=0.566) total time=   3.8s
[CV 1/3; 4/1

[CV 2/3; 10/12] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.593) total time=   3.0s
[CV 3/3; 10/12] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 3/3; 10/12] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=250, feat-selector__percentile=80, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.566) total time=   2.9s
[CV 1/3; 11/12] START clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1)
[CV 1/3; 11/12] END clf__criterion=gini, clf__max_features=sqrt, clf__n_estimators=100, feat-selector__percentile=30, preproc__preprocess__bow__ngram_range=(1, 1);, score=(train=1.000, test=0.556) total time=   2.6s
[CV 2/3; 11/12] START 

HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('preproc',
                                               Pipeline(steps=[('preprocess',
                                                                ColumnTransformer(transformers=[('bow',
                                                                                                 CountVectorizer(ngram_range=(1,
                                                                                                                              2),
                                                                                                                 tokenizer=<__main__.StemmerTokenizer object at 0x7f1acb92dca0>),
                                                                                                 'history_text'),
                                                                                                ('minmax',
                                                                            

In [54]:
search_rf.best_estimator_, 

(Pipeline(steps=[('preproc',
                  Pipeline(steps=[('preprocess',
                                   ColumnTransformer(transformers=[('bow',
                                                                    CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x7f1ac5863bb0>),
                                                                    'history_text'),
                                                                   ('minmax',
                                                                    MinMaxScaler(),
                                                                    ['intelligence_score',
                                                                     'strength_score',
                                                                     'speed_score',
                                                                     'durability_score',
                                                                     'power_score',
                

In [55]:
search_rf.score(X_test, y_test)

0.6217616580310881

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [44]:
#### Código aquí ####

# Clean nans and duplicates
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])
df_comics_no_label = df_comics_no_label.drop_duplicates(subset=['history_text'])


In [45]:
df_to_pred = df_comics_no_label[df_comics_no_label["name"].isin(["Vergil", "Gorilla Girl", "Batcow"])]

X_no_label, y_no_label = df_to_pred[[col for col in df_comics.columns if col != "alignment"]], \
            df_to_pred["alignment"].to_numpy().ravel()

In [49]:
y_pred_logreg = search_logreg.predict(X_no_label)
print(
    " Batcow:", y_pred_logreg[0], "\n", 
    "Gorilla Girl:", y_pred_logreg[1], "\n",
    "Vergil:", y_pred_logreg[2]
)

 Batcow: Good 
 Gorilla Girl: Good 
 Vergil: Good


In [53]:
y_pred_rf = search_rf.predict(X_no_label)
print(
    "Batcow:", y_pred_rf[0], "\n", 
    "Gorilla Girl:", y_pred_rf[1], "\n",
    "Vergil:", y_pred_rf[2]
)

Batcow: Good 
 Gorilla Girl: Good 
 Vergil: Good


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>